# Sandbox for pandas / ge api testing

importing necessary libraries

In [14]:
import json
import pandas as pd
import requests

inputing url for latest prices using osrs wiki ge api and getting repsonse

In [15]:
url = 'https://prices.runescape.wiki/api/v1/osrs/latest'
headers = {
    'User-Agent': 'ge_tracking',
    'From': 'onlypking'
}

response = requests.get(url, headers = headers)

storing the json repsonse in a new variable

In [16]:
json_resp = response.json()

testing to see how to format pandas dataframe

In [17]:
json_resp['data']['2']

{'high': 186, 'highTime': 1649447756, 'low': 180, 'lowTime': 1649447769}

storing the dataframe in variable df

In [18]:
df = pd.DataFrame.from_dict(json_resp['data'])

transposing for a more readable format, and storing in new variable

In [19]:
trans_df = df.transpose()

displaying the transposed df

In [20]:
trans_df

,high,highTime,low,lowTime
2,186.0,1.649448e+09,180.0,1.649448e+09
6,183629.0,1.649447e+09,179472.0,1.649448e+09
8,181513.0,1.649446e+09,178556.0,1.649448e+09
10,189989.0,1.649446e+09,185000.0,1.649448e+09
12,197382.0,1.649445e+09,186289.0,1.649448e+09
...,...,...,...,...
26788,450.0,1.649447e+09,458.0,1.649446e+09
26798,15400.0,1.649447e+09,15000.0,1.649448e+09
26801,12557.0,1.649448e+09,3001.0,1.649446e+09
26804,3999.0,1.649448e+09,1000.0,1.649444e+09


removing high and low time columns, storing in new variable clean_df

In [21]:
clean_df = trans_df.drop(['highTime', 'lowTime'], axis = 1)
clean_df

,high,low
2,186.0,180.0
6,183629.0,179472.0
8,181513.0,178556.0
10,189989.0,185000.0
12,197382.0,186289.0
...,...,...
26788,450.0,458.0
26798,15400.0,15000.0
26801,12557.0,3001.0
26804,3999.0,1000.0


labeling the item_id column as item

In [22]:
new = clean_df.rename_axis('item')
new

,high,low
item,,
2,186.0,180.0
6,183629.0,179472.0
8,181513.0,178556.0
10,189989.0,185000.0
12,197382.0,186289.0
...,...,...
26788,450.0,458.0
26798,15400.0,15000.0
26801,12557.0,3001.0


In [23]:
url_mapping = 'https://prices.runescape.wiki/api/v1/osrs/mapping' # used to get names for each item_id and current price
url_latest = 'https://prices.runescape.wiki/api/v1/osrs/latest' # used to get current high and low prices
headers = {
    'User-Agent': 'ge-tracking',
    'From': 'ge-tracker'
}

resp_map_json = requests.get(url_mapping).json()
resp_late_json = requests.get(url_latest).json()

In [63]:
df_map = pd.DataFrame.from_dict(resp_map_json)
clean_df_map = df_map.drop(['examine', 'members', 'lowalch', 'limit', 'highalch', 'icon', 'value'], axis=1).sort_values(by=['id'], ascending=True).reset_index()
cleaner_df_map = clean_df_map.drop(['index'], axis=1)
cleaner_df_map

,id,name
0,2,Cannonball
1,6,Cannon base
2,8,Cannon stand
3,10,Cannon barrels
4,12,Cannon furnace
...,...,...
3884,26788,Gold tiara
3885,26798,Catalytic talisman
3886,26801,Catalytic tiara
3887,26804,Elemental tiara


In [57]:
clean_df_late = pd.DataFrame.from_dict(resp_late_json['data']).transpose().drop(['highTime','lowTime'], axis=1)
clean_df_late.reset_index(inplace=True)
cleaner_df_late = clean_df_late.rename(columns={'index':'id'})
cleaner_df_late


,id,high,low
0,2,186.0,180.0
1,6,183629.0,179472.0
2,8,181513.0,178556.0
3,10,189989.0,185000.0
4,12,197382.0,186289.0
...,...,...,...
3820,26788,450.0,458.0
3821,26798,15400.0,15000.0
3822,26801,12557.0,3001.0
3823,26804,3999.0,1000.0


In [68]:
print(cleaner_df_map.dtypes)


id       int64
name    object
dtype: object


In [74]:
cleaner_df_late['id'] = cleaner_df_late['id'].astype(str).astype(int)
print(cleaner_df_late.dtypes)

id        int32
high    float64
low     float64
dtype: object


In [76]:
df_merge = pd.merge(cleaner_df_late, cleaner_df_map, how='outer',on='id')
df_merge

,id,high,low,name
0,2,186.0,180.0,Cannonball
1,6,183629.0,179472.0,Cannon base
2,8,181513.0,178556.0,Cannon stand
3,10,189989.0,185000.0,Cannon barrels
4,12,197382.0,186289.0,Cannon furnace
...,...,...,...,...
3886,26135,NaN,NaN,Sigil of rampage
3887,26138,NaN,NaN,Sigil of the skiller
3888,26141,NaN,NaN,Sigil of remote storage
3889,26144,NaN,NaN,Sigil of last recall
